# Procesos adjudicados
This notebook will include the analysis of all the processes that got a concract. It has a scraper for getting extra information on each process, then uses merging for having other extra data about the medicine, and then includes the analysis. 

Each row in this analysis (414) is one item: one specific combination of a medicine, in a form, concentration and comercial presentation.

## 1. Scraper - details of each contract
Firstly, to have more info on each contract, we have a scraper for getting the info on quantity, item budget and complete budget.

In [1]:
from playwright.async_api import async_playwright
from bs4 import BeautifulSoup
import pandas as pd
import asyncio

In [2]:
df = pd.read_csv("adjudicados.csv")

In [3]:
df.head(1)

,codigo,detalle_url,item,principio_activo,forma_farmaceutica,concentracion,presentacion_comercial,estado,presupuesto,provincia,canton,fecha_publicacion,descripcion
0,SICM-500-2022,https://modulocomprascorporativas.compraspubli...,DCI: LIDOCAÍNA SIN EPINEFRINA - FORMA FARMACÉU...,lidocaína sin epinefrina,sólido cutáneo parche transdérmico,5 %,caja x parche/parches,Adjudicado oferente ganador,5.0,pichincha,quito,2022-06-23 08:00:00,sistema nervioso


In [11]:
playwright = await async_playwright().start()
browser = await playwright.chromium.launch(headless=True)
page = await browser.new_page()

data = []

for i, row in df.head(5).iterrows():
    url = row["detalle_url"]
    item = {}
    item["codigo"] = row["codigo"]
    
    await asyncio.sleep(1)
    print(f"\rProcessing index: {i}, {url}", end="")
    try:
        max_attempts = 2
        for attempt in range(max_attempts):
            await page.goto(url, timeout=10000)
            await asyncio.sleep(3)
        
        try:
            await page.locator("a:has-text('Ver Resultados')").click()
            await asyncio.sleep(3)
        except:
            print(f"There is no 'Ver Resultados' in {url}")
            
        html = await page.content()
        soup = BeautifulSoup(html, "lxml")
        
        table = soup.select_one("table#rounded-corner tbody")
        row = table.find_all("td")
        if len(row) < 8:
            print(f"----- Incomplete row on {url}")
            continue
    
        item["cum_id"] = row[0].get_text(strip=True)
        item["cantidad"] = row[5].get_text(strip=True)
        item["precio_referencial"] = row[6].get_text(strip=True)
        item["subtotal"] = row[7].get_text(strip=True)


        adju_table = soup.select_one("table#tableadjudicacion")
        adju_rows = adju_table.find_all("tr")
        try:
            item["proveedor_adjudicado"] = adju_rows[0].find_all("td")[1].get_text(strip=True)
            item["valor_adjudicado"] = adju_rows[1].find_all("td")[1].get_text(strip=True)
            item["fecha_adjudicacion"] = adju_rows[3].find_all("td")[1].get_text(strip=True)
        except:
            continue
        
        data.append(item)    
        
    except:
        print(f"ERROR processing {url}")
        continue

Processing index: 4, https://modulocomprascorporativas.compraspublicas.gob.ec/ProcesoContratacion/compras/PC/informacionProcesoContratacion2.cpe?idSoliCompra=sGLIxWT-5eloFFbtFRuKxNWXmfGIylrO6hEaZXo80nk,

In [12]:
len(data)

5

In [13]:
details = pd.json_normalize(data)

In [14]:
details.head()

,codigo,cum_id,cantidad,precio_referencial,subtotal,proveedor_adjudicado,valor_adjudicado,fecha_adjudicacion
0,SICM-500-2022,N01BB02SCT220X0,184631,USD 5.000000,"USD 923,155.000000",GRUNENTHAL ECUATORIANA CIA LTDA,USD 2.990000,2022-09-05 13:11:06
1,SICM-519-2022,J05AR10SOR149X0,89200,USD 0.270000,"USD 24,084.000000",FARMAYALA PHARMACEUTICAL COMPANY S.A. FPC,USD 0.264626,2022-12-21 13:25:27
2,SICM-520-2022,J05AR03SOR189X0,189144,USD 0.438544,"USD 82,947.966336",LABORATORIOS LACFARMA S.A.,USD 0.327000,2022-12-08 16:42:39
3,SICM-503-2022,P01CB01LPR194A8,18500,USD 3.797072,"USD 70,245.832000",OFTALVIS S.A.,USD 3.560000,2022-09-05 16:36:02
4,SICM-504-2022,A07EC02SOP061X0,199725,USD 1.308369,"USD 261,313.998525",NEOETHICALS CIA. LTDA.,USD 0.850000,2022-09-21 13:16:40


In [16]:
details.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   codigo                5 non-null      object
 1   cum_id                5 non-null      object
 2   cantidad              5 non-null      object
 3   precio_referencial    5 non-null      object
 4   subtotal              5 non-null      object
 5   proveedor_adjudicado  5 non-null      object
 6   valor_adjudicado      5 non-null      object
 7   fecha_adjudicacion    5 non-null      object
dtypes: object(8)
memory usage: 452.0+ bytes


In [15]:
#I should clean details first, specially in numbers columns

In [8]:
df = pd.merge(df, details, on="codigo", how="left")

In [9]:
df.head(1)

,codigo,detalle_url,objeto,medicina,forma_farmaceutica,concentracion,presentacion_comercial,estado,presupuesto,provincia,canton,fecha_publicacion,cum_id,cantidad,precio_referencial,subtotal
0,SICM-500-2022,https://modulocomprascorporativas.compraspubli...,DCI: LIDOCAÍNA SIN EPINEFRINA - FORMA FARMACÉU...,Lidocaína Sin Epinefrina,Sólido Cutáneo Parche Transdérmico,5 %,Caja X Parche/Parches,Adjudicado oferente ganador,5.0,Pichincha,Quito,2022-06-23 08:00:00,N01BB02SCT220X0,184631,USD 5.000000,"USD 923,155.000000"


In [3]:
len(df)

414

In [24]:
df.head(1)

,code,url,objeto,medicina,forma_farmaceutica,concentracion,presentacion_comercial,estado,provincia,canton,presupuesto,fecha_publicacion
0,SICM-500-2022,https://modulocomprascorporativas.compraspubli...,DCI: LIDOCAÍNA SIN EPINEFRINA - FORMA FARMACÉU...,Lidocaína Sin Epinefrina,Sólido Cutáneo Parche Transdérmico,5 %,Caja X Parche/Parches,Adjudicado oferente ganador,Pichincha,Quito,5.0,2022-06-23 8:00:00


In [27]:
summary.columns

Index(['objeto_contractual', 'proveedor', 'cantidad',
       'monto_total_adjudicado'],
      dtype='object')

In [28]:
summary = summary.rename(columns={"cantidad": "sum_cantidad_ordenes", "monto_total_adjudicado": "sum_valor_ordenes"})

In [33]:
merged = df.merge(summary, left_on="objeto", right_on="objeto_contractual")

In [38]:
merged = merged.drop("objeto_contractual", axis=1)

In [39]:
merged.head()

,code,url,objeto,medicina,forma_farmaceutica,concentracion,presentacion_comercial,estado,provincia,canton,presupuesto,fecha_publicacion,proveedor,sum_cantidad_ordenes,sum_valor_ordenes
0,SICM-500-2022,https://modulocomprascorporativas.compraspubli...,DCI: LIDOCAÍNA SIN EPINEFRINA - FORMA FARMACÉU...,Lidocaína Sin Epinefrina,Sólido Cutáneo Parche Transdérmico,5 %,Caja X Parche/Parches,Adjudicado oferente ganador,Pichincha,Quito,5.00,2022-06-23 8:00:00,Grunenthal Ecuatoriana Cia Ltda,1552520,4642043.0
1,SICM-519-2022,https://modulocomprascorporativas.compraspubli...,DCI: LOPINAVIR + RITONAVIR - FORMA FARMACÉUTIC...,Lopinavir + Ritonavir,Sólido Oral,200 MG + 50 MG,"Caja X Frasco Dosis Personal, No Envase Hospit...",Adjudicado otros oferentes,Pichincha,Quito,0.27,2022-06-23 8:00:00,Farmayala Pharmaceutical Company S.A. Fpc,162360,42964.0
2,SICM-520-2022,https://modulocomprascorporativas.compraspubli...,DCI: TENOFOVIR + EMTRICITABINA - FORMA FARMACÉ...,Tenofovir + Emtricitabina,Sólido Oral,300 MG + 200 MG,Caja X Frasco,Adjudicado otros oferentes,Pichincha,Quito,0.44,2022-06-23 8:00:00,Laboratorios Lacfarma S.A.,189750,62048.0
3,SICM-503-2022,https://modulocomprascorporativas.compraspubli...,DCI: MEGLUMINA ANTIMONIATO - FORMA FARMACÉUTIC...,Meglumina Antimoniato,Líquido Parenteral,300 MG/ML,Caja X Ampolla-S X 5 Ml,Adjudicado oferente ganador,Pichincha,Quito,3.80,2022-06-23 8:00:00,Oftalvis S.A.,10330,36774.0
4,SICM-504-2022,https://modulocomprascorporativas.compraspubli...,DCI: MESALAZINA - FORMA FARMACÉUTICA: SÓLIDO O...,Mesalazina,Sólido Oral Polvo,1 000 MG,Caja X Sobres/Sachets,Adjudicado oferente ganador,Pichincha,Quito,1.31,2022-06-23 8:00:00,Neoethicals Cia. Ltda.,120000,102002.0


In [58]:
#Check if there are processes with no supplier...
merged[merged["proveedor"].isna()]

,code,url,objeto,medicina,forma_farmaceutica,concentracion,presentacion_comercial,estado,provincia,canton,presupuesto,fecha_publicacion,proveedor,sum_cantidad_ordenes,sum_valor_ordenes
392,SICM2-561-2022,https://modulocomprascorporativas.compraspubli...,DCI: RUXOLITINIB - FORMA FARMACÉUTICA: SÓLIDO ...,Ruxolitinib,Sólido Oral,5 MG,Caja X Blíster/Ristra,Adjudicado otros oferentes,Pichincha,Quito,48.22,2023-10-06 8:00:00,NaN,1800,81612.0
393,SICM2-451-2022,https://modulocomprascorporativas.compraspubli...,DCI: RUXOLITINIB - FORMA FARMACÉUTICA: SÓLIDO ...,Ruxolitinib,Sólido Oral,15 MG,Caja X Blíster/Ristra,Adjudicado otros oferentes,Pichincha,Quito,81.59,2023-10-06 8:00:00,NaN,2520,193334.0


In [61]:
for url in merged[merged["proveedor"].isna()]["url"]:
    print(url)

https://modulocomprascorporativas.compraspublicas.gob.ec/ProcesoContratacion/compras/PC/informacionProcesoContratacion2.cpe?idSoliCompra=nVItkprLhQu0SUhbyW3XXyhHMVCBrBftJPux9Axn24E,
https://modulocomprascorporativas.compraspublicas.gob.ec/ProcesoContratacion/compras/PC/informacionProcesoContratacion2.cpe?idSoliCompra=XGXGTZFylZFmx2faLBq2BOzllYNRv0c0_ahT_Jb5OjY,
